In [12]:
import pandas as pd
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable

%reload_ext autoreload
%autoreload 2


### Read data from the data set

Check list of all sections

In [25]:
lut = LookUpTable('Data')
sections = lut.get_sections()
pprint(list(sections)[:10])
print(len(sections))

[('4050', '48'),
 ('4050', '4060'),
 ('3120', 'Gutenberg strasse South'),
 ('6100', 'NB-East'),
 ('8003', 'Bahnhofstrasse'),
 ('23', '4020'),
 ('12', '4020'),
 ('1022', '1040'),
 ('8604', '8605'),
 ('6070', '6080')]
236


/home/kolusask/TUM/IDP/src/data/extract.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.lookup_table = self.lookup_table[


Extract list of detectors for each section

In [26]:
int_det = lut.get_detectors_per_section()
int_det

,Start,End,Detectors
0,4050,48,[2(DB1)]
1,48,4050,[6(DB1)]
2,4050,4060,"[1(DA1), 2(DA2), 3(DA3)]"
3,4060,4050,"[3(DC1), 4(DC2), 5(DCL1)]"
4,3120,Gutenberg strasse South,[]
...,...,...,...
467,Network Boundary,6050,"[3(DB1A), 4(DB1B)]"
468,12,4010,"[1(DA1), 2(DA2)]"
469,4010,12,"[4(DC1), 3(DC2)]"
470,3130,3150,[]


Extract counts for all sections

In [33]:
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2021, 8, 1)

ddp = DetectorDataProvider('Data')

int_det['Counts'] = int_det.apply(lambda sec: ddp.get_counts_entering_section(sec['End'], sec['Detectors'], START_DATE, END_DATE), axis=1)

int_det.to_hdf(out_path('int_det.hdf'), 'int_det')
int_det

/tmp/ipykernel_52211/1263797608.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Start', 'End', 'Detectors', 'Counts'], dtype='object')]

  int_det.to_hdf(out_path('int_det.hdf'), 'int_det')


,Start,End,Detectors,Counts
0,4050,48,[2(DB1)],"[0.0, 1.0, 3.0, 7.0, 1.0, 1.0, 1.0, 2.0, 2.0, ..."
1,48,4050,[6(DB1)],"[1.0, 0.0, 2.0, 2.0, 0.0, 2.0, 1.0, 1.0, 0.0, ..."
2,4050,4060,"[1(DA1), 2(DA2), 3(DA3)]","[1.0, 10.0, 2.0, 5.0, 4.0, 6.0, 1.0, 3.0, 4.0,..."
3,4060,4050,"[3(DC1), 4(DC2), 5(DCL1)]","[2.0, 5.0, 4.0, 11.0, 2.0, 2.0, 6.0, 2.0, 5.0,..."
4,3120,Gutenberg strasse South,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
467,Network Boundary,6050,"[3(DB1A), 4(DB1B)]","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
468,12,4010,"[1(DA1), 2(DA2)]","[4.0, 0.0, 2.0, 7.0, 1.0, 2.0, 4.0, 2.0, 1.0, ..."
469,4010,12,"[4(DC1), 3(DC2)]","[3.0, 6.0, 2.0, 2.0, 4.0, 9.0, 0.0, 0.0, 1.0, ..."
470,3130,3150,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [35]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, out_path('mat_q.pt'))
mat_q.shape

torch.Size([20352, 472])